# Lab 10: Computational skills moving forward
This course probably won't be the last time you use Python for your scientific work.  Our goal today is to get you set up for future use cases: 
1. finding, accessing, and loading new data, and 
2. setting up a new conda environment.

I'll provide some resources, we'll work through an example together, and we can discuss any questions you have along the way.

---
## Part 1. Accessing public data
You have processed a few publicly available datasets, some of which were served nicely in global gridded format (ERA5 data in Lab 2, 6, and 7 and Assignment 5) and some of which you wrassled into shape from trickier formats (TAO/TRITON data in Lab 8; NCEI data in class).  These data were primarily in **NetCDF** and **CSV** formats.  We used `xarray` and `pandas` to process them.

There are many public repositories where you can find data for other analyses you might want to do.  For example:
- the [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/#!/home) for climate datasets, including ERA5, that were produced by European institutions
- the [National Snow & Ice Data Center](https://nsidc.org/data/explore-data) for cryosphere data, mostly US-produced
- regional focus repositories such as [NSF Arctic Data Center](https://arcticdata.io/catalog/data)
...


### Example: CMIP6 projections
One thing you might like to do now is explore future climate projections.  You can retrieve these from Copernicus.
1. Click the Copernicus Climate Data Store link above.
2. Click "login/register" and follow the steps to make an account.
3. Return to the site and enter "CMIP6" in the search bar.  This corresponds to the sixth, most recent, phase of the Coupled Model Intercomparison Project
4. Click "CMIP6 climate projections" and read about the data that is included.
5. Click "Download data" and fill out the form to select monthly data, for SSP2-4.5 (for example), for whichever variable you want to analyse.  I will show sea ice concentration and thickness for a single GCM (CanESM2) below.
6. Unzip the data and put it in the folder where you want to access it.  Update the path below.

In [ ]:
# Import the tools (stolen from Lab 6):
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size

In [ ]:
fpath = '/Users/lizz/Documents/GitHub/climdyn-labs/data/dataset-projections-cmip5-monthly-single-levels-71df9c03-3f6e-4f30-a7be-07c0a67f6460/'
f_sic = fpath+'sic_OImon_CanESM2_rcp45_r1i1p1_200601-210012.nc'
f_sit = fpath+'sit_OImon_CanESM2_rcp45_r1i1p1_200601-210012.nc'

In [ ]:
ds1 = xr.open_dataset(f_sic) ## open sea ice concentration first
ds1

In [ ]:
ds2 = xr.open_dataset(f_sit) ## now inspect sea ice thickness
ds2

#### Global map plot inspection

Let's compute a time average and plot it on a map, as we have done before.

In [ ]:
sic_tavg = ds1.sic.mean(dim='time')
ax = plt.axes(projection=ccrs.Robinson())
sic_tavg.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Blues')
ax.coastlines(); ax.gridlines();

This makes sense.  There is highest average sea ice concentration near the poles, and zero sea ice concentration at the equator.  Let's inspect a NH winter month: 

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
ds1.sic.sel(time='2010-03-16').plot(ax=ax, 
                                    transform=ccrs.PlateCarree(), 
                                    cmap='Blues')
## notice how I had to choose exactly one of the times 
## shown in the 'time' dimension
ax.coastlines(); ax.gridlines();

And now let's see the end of NH summer / SH winter:

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
ds1.sic.sel(time='2010-09-16').plot(ax=ax, 
                                    transform=ccrs.PlateCarree(), 
                                    cmap='Blues')
## notice how I had to choose exactly one of the times 
## shown in the 'time' dimension
ax.coastlines(); ax.gridlines();

Looks about like what we'd expect.  The second dataset I've read in, `ds2`, is sea ice thickness output by the same model.  Let's see:

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
ds2.sit.sel(time='2010-09-16').plot(ax=ax, 
                                    transform=ccrs.PlateCarree(), 
                                    cmap='Blues')
## notice how I had to choose exactly one of the times 
## shown in the 'time' dimension
ax.coastlines(); ax.gridlines();

We are well equipped to answer questions like: What's the maximum sea ice thickness in the early part of the century?

In [ ]:
ds2.sit.sel(time=slice('2006','2020')).max()

Looks like 3.8 meters is the maximum sea ice thickness in the early 21st century.

Let's see what this model projects for the future.  I asked for the SSP2-4.5 scenario; remember that these outputs will be different depending on the socioeconomic pathway and the model used to project them.  With that in mind...

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
ds1.sic.sel(time='2099-09-16').plot(ax=ax, 
                                    transform=ccrs.PlateCarree(), 
                                    cmap='Blues')
## notice how I had to choose exactly one of the times 
## shown in the 'time' dimension
ax.coastlines(); ax.gridlines();

This model projects an ice-free Arctic summer at the end of the century.  What's the max sea ice thickness at the end of the century?

In [ ]:
ds2.sit.sel(time=slice('2085','2100')).max()

Looks like about a 25% reduction in the maximum ice thickness.  So in addition to the loss of concentration, it looks like sea ice is not getting as thick (or persisting long enough to get thick) as it did in the past.

As we close our time together, you are well-equipped to analyse and compare projections like these!  We will not spend time doing it now, but I am happy to chat about any analysis you can dream up. :)

---
## Part 2. Setting up a new conda environment
You will probably need slightly different computational tools in the future, beyond the environment we've used for class.  Now that you have the Anaconda Python distribution set up, it is straightforward to set up new environments that won't interfere with your existing packages.

You can see exactly how easy on the [conda cheat sheet](https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf), which I use *all the time*.

### Example: an environment with QGIS
Let's say you are doing some geospatial analysis and want to use GIS.  

QGIS is open-source and can be installed directly with conda, and we can put it in its own environment so it doesn't touch anything in the ECSC 202 environment.

First we make a new environment and give it some sensible name.  **In the terminal:**
```
    conda create --name qgis_example
```
And activate it:
```
    conda activate qgis_example
```

We can install QGIS from the `conda-forge` channel:
```
    conda install -c conda-forge qgis
```

How did I know to give the `-c conda-forge` command? I suspected QGIS might be on conda-forge rather than the default conda channel because it is not a "core" Python package, but I confirmed it by looking [on StackExchange](https://gis.stackexchange.com/questions/119495/does-qgis-work-with-anaconda)!  This is not the kind of thing I expect anyone to just remember offhand.

Once the installation completes, we can launch QGIS from our example environment by simply typing `qgis`.

You can find conda-based installation instructions for many open-source software packages online.  Remember: all you need is a new environment, not a whole new conda installation.  

#### Help! I messed up my environment! 
If you happen to mess up your ECSC202 environment, it's easy to remove and reinstall it from our `environment.yml` file following the conda cheat sheet.  

If you're nervous -- like if it's month 8 of your 9-month thesis and you need to install some finicky package -- I advise that you follow the instructions to **save your active environment to an environment.yml file** before you mess around with something new.  Then you will know how to get back to the working environment if something goes wrong.

Conda is *not* touching any of your Jupyter notebooks or Python scripts.  They will all be preserved even if you remove and reinstall the environment.  Conda is only managing the packages (anything we `import`) in the environment.

---
### Endnotes:
- This notebook last updated 7 May 2024 by Lizz Ultee